In [1]:
import sys
import os
import wandb
import torch
import numpy as np

import pyro
import pyro.distributions as dist
import pyro.infer.mcmc as mcmc

from ml_collections import ConfigDict

# Add the parent directory to the sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from files.training import train_dga
from files.train_elliptic import train_elliptic,generate_data,samples_param


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/s2113174/anaconda3/envs/fenicsx-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Burgers

In [2]:

def get_burgers_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Burgers_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "burgers"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"burgers":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    # config.model.input_dim = 3
    # config.model.hidden_dim = 20
    # config.model.num_layers = 2
    # config.model.out_dim = 1
    # config.model.activation = "tanh"
    config.model.layers = [3] + 3*[20] + [1]

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    #config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":256,"exclude_last_n":100})

    # Training settings
    config.seed = 42
    config.learning_rate = 0.001
    config.decay_rate = 0.95
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 0.75
    # config.weights_update = 250
    config.scheduler_step = 50

    config.batch_size = 150
    
    return config

In [3]:
N = [100,1500]

#Loop through each sample size
for batch_size in N:
    print(f"Running training with {batch_size} samples...")
    config = get_burgers_config()
    config.wandb.name = f"Burgers_MDNN_s{batch_size}"
    config.batch_size = batch_size  # Update the samples in the config
    pinn_nvs = train_dga(config, device=device)
    print(f"Completed training with {batch_size} samples.")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Running training with 100 samples...


wandb: Currently logged in as: christian-jb07 (christian-jb07-univeristy-of-edinburgh). Use `wandb login --relogin` to force relogin


epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇██
learning_rate,██████████████████████████████████▅▅▄▃▂▁
loss_burgers,██▇▇▅▄▅▄▂▅▄▃▅▃▃▂▃▂▄▃▂▂▂▂▂▂▁▂▁▂▁▁▂▂▁▃▁▁▂▁
loss_computation_time,▂▄▃▂▂▂▂▁▁▂█▁▁▃▂▁▂▅▅▁▅▅▂▄▆▄▃▃▃▂▃▂▆▃▂▄▂▃▂▂
loss_ubcl,█▃▃▃▃▃▄▃▃▃▃▂▂▂▂▁▁▃▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▅▃▃▃▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_uic,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇██▇▄▃▃▂▂▂▂▃▃▂▂▂▂▃▁▂▂▂▂▁▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,0.00028
loss_burgers,0.01148


Completed training with 100 samples.
Running training with 1500 samples...


epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇███
learning_rate,███████████████████████████████▄▃▃▂▂▂▂▂▁
loss_burgers,█▇▇▇▆▄▄▄▄▃▄▃▂▃▃▂▂▂▃▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
loss_computation_time,▂▂▄▁▂▂▂▂▂▂▃▅█▆▅▂▅▄▃▄▄▂▅▂▄█▃▆▂▃▄▅▃▂▂▂▄▃▂▂
loss_ubcl,█▇▅▃▃▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_uic,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,0.00028
loss_burgers,0.00927


Completed training with 1500 samples.


# Heat Eq

In [4]:

def get_heat_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "PDEs-training"
    wandb.name = "Heat_MDNN"
    wandb.tag = None

    # General settings
    config.dga = "heat"
    config.nn_model = "NN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"heat":1,"uic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    # config.model.input_dim = 3
    # config.model.hidden_dim = 20
    # config.model.num_layers = 2
    # config.model.out_dim = 1
    # config.model.activation = "tanh"
    config.model.layers = [3] + 3*[20] + [1]

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    # config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":20,"exclude_last_n":1})

    # Training settings
    config.seed = 42
    config.learning_rate = 0.001
    config.decay_rate = 0.95
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 0.5
    # config.weights_update = 250
    config.scheduler_step = 50

    config.batch_size = 1000
    
    return config

In [5]:
N = [1000,3000]

#Loop through each sample size
for batch_size in N:
    print(f"Running training with {batch_size} samples...")
    config = get_heat_config()
    config.wandb.name = f"Heat_MDNN_s{batch_size}"
    config.batch_size = batch_size  # Update the samples in the config
    pinn_nvs = train_dga(config, device=device)
    print(f"Completed training with {batch_size} samples.")

Running training with 1000 samples...


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█████
learning_rate,████████████████████▄▄▄▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁
loss_computation_time,▅▃▂▁▁▁▁▁▁▁▁▅▁▁▅▆▁▁▁▁▃▁▁▂█▅▇▁▄▆█▂▁▅▁▂▂▅▂▅
loss_heat,▁▁▁▁▂▃▆█▇█▇█▆██▆▆▄▄▆▅▆▆▅▅▄▄▅▄▅▆▄▅▄▄▅▆▅▅▄
loss_ubcl,▁▁▆█▃▃▄▄▃▅▅▃▄▆▅▅▆▅▅▇▄▆▆▆▆▆▅▆▇▇▆▆▆▆▆▆▆▇▆▆
loss_ubcr,▁▂▂▄▇▅▄▄▂▄▄▇▄▅▆▅█▆▇▇▆▇▆▇▆▇▆▇▇▆▇▇▆▆▇▇▇▇▇▇
loss_uic,███▅▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,███▇▅▄▄▄▃▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00177


Completed training with 1000 samples.
Running training with 3000 samples...


epoch,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇████
learning_rate,████████████████████████▇▅▅▅▅▄▃▃▂▂▂▂▂▂▁▁
loss_computation_time,▁▃▂█▂▄▄█▄▃▄▄▄▅▃▆▄▆▃▄▃▃▄▄▃▃▃▂▃▄▃▅▆▆▆▃▂▃▄▅
loss_heat,▁▁▃█▇▆▆▆▅▅▅▅▄▄▆▅▅▄▄▅▄▄▅▅▄▅▄▄▅▄▄▄▄▄▅▅▅▄▅▄
loss_ubcl,▃▃▃██▅▆▅▅▅▆▅▅▃▃▃▂▃▃▂▁▁▂▁▁▂▂▁▁▁▁▂▁▁▁▂▂▁▂▁
loss_ubcr,▂▁▁▁▁▆█▅▄▃▄▃▄▄▃▄▄▃▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
loss_uic,██████▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00747


Completed training with 3000 samples.


# Elliptic

In [ ]:

def get_elliptic_config():
    config = ConfigDict()

    # Weights & Biases
    config.wandb = wandb = ConfigDict()
    wandb.project = "Elliptic-training"
    wandb.name = "MDNN"
    wandb.tag = None

    # General settings
    config.nn_model = "MDNN"  # Options: "NN", "WRF", "MDNN"
    config.lambdas = {"elliptic":1, "ubcl":1, "ubcr":1}

    # Model-specific settings
    config.model = ConfigDict()
    config.model.input_dim = 3
    config.model.hidden_dim = 20
    config.model.num_layers = 2
    config.model.out_dim = 1
    config.model.activation = "tanh"

    # Weight-Random-Factorization
    #config.reparam = ConfigDict({"type":"weight_fact","mean":1.0,"stddev":0.1})

     # Periodic embeddings
    #config.model.period_emb = ConfigDict({"period":(1.0, 1.0), "axis":(0, 1) })

    # Fourier embeddings
    #config.model.fourier_emb = ConfigDict({"embed_scale":1,"embed_dim":256,"exclude_last_n":100})

    # Config KL parameters
    config.nparameters = 2

    # Training settings
    config.seed = 42
    config.learning_rate = 0.001
    config.decay_rate = 0.95
    # config.alpha = 0.9  # For updating loss weights
    config.epochs = 5000
    config.start_scheduler = 0.5
    # config.weights_update = 250
    config.scheduler_step = 50

    config.samples = 5000
    config.batch_size = 150

    
    return config

In [ ]:
N = [150,2500]

#Loop through each sample size
for samples in N:
    print(f"Running training with {samples} samples...")
    config = get_elliptic_config()
    config.wandb.name = f"MDNN_s{samples}"
    config.samples = samples  # Update the samples in the config
    pinn_nvs = train_elliptic(config, device=device)
    print(f"Completed training with {samples} samples.")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Running training with 150 samples...


wandb: Currently logged in as: christian-jb07 (christian-jb07-univeristy-of-edinburgh). Use `wandb login --relogin` to force relogin


epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇█████
learning_rate,██████████████████▇▇▆▆▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
loss_computation_time,▄▂▆▇▁▄█▄▂▆▃▃▃▂▆▂▁▂▂▂▃▂▄▂▃▂▂▂▃▂▅▅▁▃▁▁▃▁▂▂
loss_elliptic,█▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00295


Completed training with 150 samples.
Running training with 2500 samples...


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇█
learning_rate,███████████████████▇▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
loss_computation_time,█▂▂▁▂▂▁▃▁▂▂▁▁▃▁▄▁▄▂▂▃▃▄▃▄▁▃▃▃▂▁▂▂▄▄▆▁▂▄▃
loss_elliptic,█▇▃▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcl,█▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_ubcr,█▇▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▅▃▂▂▁▁▂▂▂▁▁▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,4999
learning_rate,8e-05
loss_computation_time,0.00117


Completed training with 2500 samples.
